# Table of Contents
 <p><div class="lev1 toc-item"><a href="#0.-Data-exploration" data-toc-modified-id="0.-Data-exploration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>0. Data exploration</a></div><div class="lev2 toc-item"><a href="#0.1-Load-in-the-schema-of-the-data." data-toc-modified-id="0.1-Load-in-the-schema-of-the-data.-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>0.1 Load in the schema of the data.</a></div><div class="lev2 toc-item"><a href="#0.2-Import-sample-of-the-data" data-toc-modified-id="0.2-Import-sample-of-the-data-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>0.2 Import sample of the data</a></div><div class="lev2 toc-item"><a href="#0.3-Data-stats" data-toc-modified-id="0.3-Data-stats-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>0.3 Data stats</a></div><div class="lev3 toc-item"><a href="#Total-Number-of-Tweets" data-toc-modified-id="Total-Number-of-Tweets-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Total Number of Tweets</a></div><div class="lev3 toc-item"><a href="#Column-formats" data-toc-modified-id="Column-formats-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Column formats</a></div><div class="lev3 toc-item"><a href="#Column-Descriptions" data-toc-modified-id="Column-Descriptions-133"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Column Descriptions</a></div><div class="lev3 toc-item"><a href="#NaN-Cells" data-toc-modified-id="NaN-Cells-134"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>NaN Cells</a></div><div class="lev3 toc-item"><a href="#Number-of-Users-in-Sample-Data" data-toc-modified-id="Number-of-Users-in-Sample-Data-135"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Number of Users in Sample Data</a></div><div class="lev3 toc-item"><a href="#Number-of-Tweets-per-User" data-toc-modified-id="Number-of-Tweets-per-User-136"><span class="toc-item-num">1.3.6&nbsp;&nbsp;</span>Number of Tweets per User</a></div><div class="lev4 toc-item"><a href="#Observation" data-toc-modified-id="Observation-1361"><span class="toc-item-num">1.3.6.1&nbsp;&nbsp;</span><em>Observation</em></a></div><div class="lev3 toc-item"><a href="#Convert-'createdAt'-to-datetime-type" data-toc-modified-id="Convert-'createdAt'-to-datetime-type-137"><span class="toc-item-num">1.3.7&nbsp;&nbsp;</span>Convert 'createdAt' to datetime type</a></div><div class="lev4 toc-item"><a href="#Observation" data-toc-modified-id="Observation-1371"><span class="toc-item-num">1.3.7.1&nbsp;&nbsp;</span><em>Observation</em></a></div><div class="lev3 toc-item"><a href="#Remove-Urls-from-tweets" data-toc-modified-id="Remove-Urls-from-tweets-138"><span class="toc-item-num">1.3.8&nbsp;&nbsp;</span>Remove Urls from tweets</a></div><div class="lev4 toc-item"><a href="#Observation" data-toc-modified-id="Observation-1381"><span class="toc-item-num">1.3.8.1&nbsp;&nbsp;</span><em>Observation</em></a></div><div class="lev3 toc-item"><a href="#Language-Usage" data-toc-modified-id="Language-Usage-139"><span class="toc-item-num">1.3.9&nbsp;&nbsp;</span>Language Usage</a></div><div class="lev4 toc-item"><a href="#Observation" data-toc-modified-id="Observation-1391"><span class="toc-item-num">1.3.9.1&nbsp;&nbsp;</span><em>Observation</em></a></div><div class="lev3 toc-item"><a href="#0.3.1-Percentage-with-geolocation" data-toc-modified-id="0.3.1-Percentage-with-geolocation-1310"><span class="toc-item-num">1.3.10&nbsp;&nbsp;</span>0.3.1 Percentage with geolocation</a></div>

> **Purpose of this notebook: **

> Before diving to the huge datasets provided by Swisscom, need to understand it and understand what we want to do with it. So we will use this notebook with a small sample of the datasets, to simulate the steps of our project. The final goal is to create a python scipt, so we just need to launch it on the cluster to obtain the desired results.

**Import Packages**

In [57]:
import pandas as pd
import numpy as np
import csv

# plots
from matplotlib.pyplot import *
matplotlib.style.use('ggplot')

from pylab import *

from pandas import Series
import re

# language detection
from langdetect import detect

In [58]:
print("Pandas version {}".format(pd.__version__))

Pandas version 0.19.2


In [59]:
#increase the max column width displayed from the default 50
pd.set_option('display.max_colwidth', 200)

# 0. Data exploration

## 0.1 Load in the schema of the data.
***

Note that *schema.txt* file given in the epfl cluster was manually modified in order to import it easily with pandas.

In [60]:
# read schema 
schema_rawfile = pd.read_csv("twitter-swisscom/schema_home.txt", header=None, sep='\s+')
# extract schema columns
twitter_data_columns = schema_rawfile[1].values

print("Nbr features:",len(twitter_data_columns))
list(twitter_data_columns)

Nbr features: 20


['id',
 'userId',
 'createdAt',
 'text',
 'longitude',
 'latitude',
 'placeId',
 'inReplyTo',
 'source',
 'truncated',
 'placeLatitude',
 'placeLongitude',
 'sourceName',
 'sourceUrl',
 'userName',
 'screenName',
 'followersCount',
 'friendsCount',
 'statusesCount',
 'userLocation']

## 0.2 Import sample of the data
***

We will not work directly with the entire datasets. Instead, we prepare our code and function here, with a small sample of the data. Then we create a script to be executed in the EPFL cluster.


**First step:** "Clean" the tsv file. The files we are given are in a *.tsv* format: there is one tweet per line and each features within a tweet are separated by a tab. Unfortunately, some tweets text contain break line (*\n*). If we try to import them like that, python will take those break line *\n* as a new line, therefore as a new tweet. 

But the break line inside the tweets are preceed by a backslash. Therefore, we run a bash command to replace all combination of *\ + \n* by a simple white space.

>     sed -e :a -e '/\\$/N; s/\\\n/ /; ta' twitter-swisscom/sample.tsv  > data_clean/sample.tsv

**Second step**: The file *data_clean/sample.tsv* has the following attributes:

    *application/octet-stream; charset=binary*

Therefore, we cannot load it *as is* into a pandas Dataframe. We will convert it to a simple *text/plain* with encoding *utf-8*, that pandas can perfectly handle.

>     cat data_clean/sample.tsv | tr -d '\0' > data_clean/sample_formatted.tsv

**Third step**: Now we can import the data to a pandas dataframe.

In [61]:
sample_data = pd.read_table(r"data_clean/sample_formatted.tsv",sep='\t', quoting=csv.QUOTE_NONE, header=None, names=twitter_data_columns)
sample_data.head()

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,776522983837954049,735449229028675584,2016-09-15 20:48:01,se lo dici tu... https://t.co/x7Qm1VHBKL,\N,\N,51c0e6b24c64e54e,\N,1,NaN,46.0027,8.96044,Twitter for iPhone,http://twitter.com/#!/download/iphone,plvtone filiae.,hazel_chb,146,110,28621,Earleen.
1,776523000636203010,2741685639,2016-09-15 20:48:05,https://t.co/noYrTnqmg9,\N,\N,4e7c21fd2af027c6,\N,1,NaN,46.8131,8.22414,Twitter for iPhone,http://twitter.com/#!/download/iphone,samara,letisieg,755,2037,3771,Suisse
2,776523045200691200,435239151,2016-09-15 20:48:15,"@BesacTof @Leonid_CCCP Tu dois t'engager en signant précisément:""Je partage les valeurs républicaines de la droite et du centre 1/2...",\N,\N,12eb9b254faf37a3,776522113859608576,5,NaN,47.2010,5.94082,Twitter for Android,http://twitter.com/download/android,lebrübrü❤,lebrubru,811,595,30191,Fontain
3,776523058404290560,503244217,2016-09-15 20:48:18,@Mno0or_Abyat اشوف مظاهرات على قانون العمل الجديد انا طلعت الصباح من هناك مافيه شي بس اشوف الأخبار قبل شوي المشكلة راجع بعد يومين !!,\N,\N,30bcd7f767b4041e,776521597515624448,1,NaN,45.8011,6.16552,Twitter for iPhone,http://twitter.com/#!/download/iphone,عبدالله القنيص,bingnais,28433,417,12262,Shargeyah
4,776523058504925185,452805259,2016-09-15 20:48:18,Greek night #geneve (@ Emilios in Genève) https://t.co/sEplW0Mcyz,6.14414,46.1966,c3a6437e1b1a726d,\N,3,NaN,46.2048,6.14319,foursquare,http://foursquare.com,Alkan Şenli,Alkanoli,204,172,3390,İstanbul/Burgazada


## 0.3 Data stats

***
### Total Number of Tweets

We have **_8790_** entries in our dataframe and each row contains **_20_** columns.

In [62]:
print(len(sample_data))
print(len(sample_data.columns))

8790
20


### Column formats

In [63]:
sample_data.dtypes

id                  int64
userId              int64
createdAt          object
text               object
longitude          object
latitude           object
placeId            object
inReplyTo          object
source              int64
truncated         float64
placeLatitude     float64
placeLongitude    float64
sourceName         object
sourceUrl          object
userName           object
screenName         object
followersCount      int64
friendsCount        int64
statusesCount       int64
userLocation       object
dtype: object

### Column Descriptions
We've gathered more detailed dscription of each attribute from the [Twtter API](https://dev.twitter.com/overview/api/users)

In [64]:
data_description = pd.read_csv("twitter-swisscom/schema_description.txt", header=None, names=['Attribute', 'Description'], sep='\s\s+', engine='python', index_col=0)
data_description

,Attribute,Description
1,id,The integer representation of the unique identifier for this Tweet.
2,userId,The integer representation of the unique identifier for this User.
3,createdAt,"UTC time when this Tweet was created. Example: ""created_at"":""Wed Aug 27 13:08:45 +0000 2008"""
4,text,The actual UTF-8 text of the status update.
5,longitude,Nullable Represents the geographic location of this Tweet as reported by the user or client application.
6,latitude,Nullable Represents the geographic location of this Tweet as reported by the user or client application.
7,placeId,"ID representing this place. Places are specific, named locations with corresponding geo coordinates. They can be attached to Tweets by specifying a place_id when tweeting. Tweets associated with p..."
8,inReplyTo,"Nullable If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet’s ID."
9,source,None
10,truncated,"Indicates whether the value of the text parameter was truncated, for example, as a result of a retweet exceeding the 140 character Tweet length. Truncated text will end in ellipsis, like this ... ..."


### NaN Cells
***
The **text** column (tweet content) contains NaN Value. We have no choice but to remove them because without the content we won't be able to proceed and perform sentiment analysis. As for **truncated** and **userName**, we don't worry much as truncated is just a flag indicating whether the content is above 140 characters and userId is better and more unique than userName.

In [65]:
sample_data.isnull().any()

id                False
userId            False
createdAt         False
text               True
longitude         False
latitude          False
placeId           False
inReplyTo         False
source            False
truncated          True
placeLatitude     False
placeLongitude    False
sourceName        False
sourceUrl         False
userName           True
screenName        False
followersCount    False
friendsCount      False
statusesCount     False
userLocation      False
dtype: bool

In [66]:
# Create a true/false list then map to the original dataframe
NaN_text = sample_data.text.isnull()
sample_data = sample_data[~ NaN_text]

In [67]:
print("Number of entries missing content - ", NaN_text.sum())
print("Entries in sample_data dataframe - ", sample_data.text.count())
print("Percentage of missing data", 93/8790)

Number of entries missing content -  93
Entries in sample_data dataframe -  8697
Percentage of missing data 0.010580204778156996


### Number of Users in Sample Data
***

All **8697** entries in our dataframe have an unique id. However there are **2739** unique users in our data. 

In [68]:
total_tweets = len(pd.unique(sample_data.id.ravel()))
total_unique_user = len(pd.unique(sample_data.userId.ravel()))

print("Percentage of {}(users) / {}(tweets) = {}".format(total_unique_user, total_tweets, total_unique_user/total_tweets))

Percentage of 2739(users) / 8697(tweets) = 0.31493618489134184


### Number of Tweets per User 
***

In [69]:
def f(x):
    return Series(dict(Number_of_tweets = x['text'].count(),
                       User_name = x['userName'].min(),
                       tweet_id = x['id'].min(),
                       ))

In [70]:
tweet_count = sample_data.groupby('userId').apply(f)
tweet_count = tweet_count.sort_values('Number_of_tweets', axis=0, ascending=False)
tweet_count.head()

,Number_of_tweets,User_name,tweet_id
userId,,,
766674630014857217,217.0,Poke Luzern,7.765256e+17
758247740019126272,186.0,ElnellaFeverOfficial,7.765501e+17
735449229028675584,111.0,plvtone filiae.,7.765230e+17
570608126,93.0,Hervé M,7.766640e+17
866661589,89.0,ana,7.765258e+17


#### *Observation*

More than 50% of the user only had 1 tweet in our sample data, total avg is 3 tweets per user.

In [71]:
tweet_count.Number_of_tweets.describe()

count    2739.000000
mean        3.175246
std         8.318616
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max       217.000000
Name: Number_of_tweets, dtype: float64

### Convert 'createdAt' to datetime type
***

The **createdAt** column is of type String and contains the UTC time when this Tweet was created. However we want to convert this to the datatime Python type for the ease of manipulation in future analysis.

In [72]:
sample_data.dtypes[:3]

id            int64
userId        int64
createdAt    object
dtype: object

In [73]:
# Convert the type
sample_data['createdAt'] = pd.to_datetime(sample_data['createdAt'])
sample_data.dtypes[:3]

id                    int64
userId                int64
createdAt    datetime64[ns]
dtype: object

In [74]:
# We can now sort our dataframe based on createdAt column
sample_data = sample_data.sort_values('createdAt', axis=0, ascending=False)

In [75]:
sample_data.createdAt.describe()

count                    8697
unique                   8070
top       2016-09-16 08:25:00
freq                        4
first     2016-09-15 20:48:01
last      2016-09-16 16:35:19
Name: createdAt, dtype: object

#### *Observation*

There are *8070* unique datatimes in our dataframe, and all of our tweets are created between *2016-09-15 20:48:01* and *2016-09-16 08:25:00*

### Remove Urls from tweets
***

We want to remove urls from the content of the tweet because it gives us no information about the sentiment and the package we use to detect what language the tweet is written can't process a single url as the text has no feature to analyse.

In [76]:
for index, row in sample_data.iterrows():
    # match url in string and replace with "" empty string
    t = re.sub(r"http\S+", "", row['text'])
    if (t.strip() == ""): # remove the row if the result is empty after stripping
        sample_data = sample_data.drop(index)
    else:
        sample_data.set_value(index, "text", t)

#### *Observation*

From our inspection, most urls are removed from the tweet. As a result we have shrunk the dataset to *8556* entries. **Note**: Although Urls don't contain any feature for sentiment analysis, but it could be the link to connect different tweets together (different user discussing/expressing their thought about the same topic)

### Language Usage
***

We use [langdetect](https://pypi.python.org/pypi/langdetect) python package to determine what language is used for the tweet. Of course come tweets are written in multiple languages, this library returns the most probable language after analysis.

In [144]:
TOTAL_ENTRIES = len(sample_data)

TOTAL_ENTRIES

8556

In [78]:
sample_data['language'] = ''

for index, row in sample_data.iterrows():
    lang = "None"
    try:
        lang = detect(row['text'])
    except:
        pass
    sample_data.set_value(index, "language", lang)

In [140]:
def d(x):
    return Series(dict(Number_of_tweets = x['text'].count(),
                       Percentage = x['text'].count() / TOTAL_ENTRIES * 100,
                       ))

In [143]:
lang_usage = sample_data.groupby('language').apply(d).sort_values('Percentage', axis=0, ascending=False)

lang_usage

,Number_of_tweets,Percentage
language,,
en,2391.0,27.945302
fr,1680.0,19.635344
de,1197.0,13.990182
it,774.0,9.046283
ar,383.0,4.476391
es,327.0,3.821879
pt,251.0,2.933614
tr,161.0,1.881720
id,140.0,1.636279


#### *Observation*

**English** is the most used language written ***27.9%*** of total tweets followed by ***19.6%*** of **French**, ***13.9%*** of **German**, ***9.0%*** of **Italian**, ***4.4%*** of **Arabic** and ***3.8%*** of **Spanish**. 

### 0.3.1 Percentage with geolocation

In [ ]:
long = sample_data.longitude
long_not_null = (long != r'\N').sum()
print("Percentage of data with longitude info:", 100*(long_not_null/len(sample_data)))
print("Nbr: ", long_not_null)

In [ ]:
lat = sample_data.latitude
lat_not_null = (lat != r'\N').sum()
print("Percentage of data with latitude info:", 100*(lat_not_null/len(sample_data)))
print("Nbr: ", lat_not_null)

In [ ]:
long_not_null = long!=r'\N'
lat_not_null = lat!=r'\N'
diff = (long_not_null != lat_not_null).any()
print("Is there null long/lat with valid lat/long ? :", diff)

That's for the latitude/longitude... what ? The user ? 
Because we also have the fields ***placeLongitude*** and ***placeLatitude***.

From the twitter api:
> Places are specific, named locations with corresponding geo coordinates. They can be attached to Tweets by specifying a place_id when tweeting. Tweets associated with places are not necessarily issued from that location but could also potentially be about that location.

Since we want to map the sentiment of tweet with a given location, we will use fields ***placeLongitude*** and ***placeLatitude*** for this project.

In [ ]:
xx = sample_data.groupby(['placeId']).size()
xx.sort(ascending=False)
xx